In [183]:
import bs4
import pandas as pd
import numpy as np
from src.util.util_dicts import nba_teams_post_2000, home_map
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import random

## Scraping Player logs

#### Oct 26th Thinking

Did not get as much done yesterday, now have NBA players list as CSV, full and post 2000. Goal for today:
- Pull and clean advanced stats
- Append to player game log df (s)
- Git cleaning (s)
- Create BQ table and write NBA player list (s)
- Create BQ table and write post 2000 nba (s)

#### Oct 25th Thinking

Lots of progress yesterday, need to start thinking about saving data to cloud and cleaning up repository
For today, consider doing:
- Clean up repository
- Investigate advanced player stats
- Write all nba/aba players to table (consider keeping additional context)
- Investigate edge cases in names (Marvin Bagley III, Dennis Smith Jr). Know names with more than one ' ' will be an issue
- Start thinking about looping over player names, perhaps one letter at a time, have a csv for each letter that can easily be opened and added to BQ

#### Oct 24th Thinking

- Got way to search player game logs by season, next need to clean up the returned table into a single clean df
- Showing need to extract all selenium related functions into their own specific python file in the future (selenium_helper.py)


#### Oct 23rd Thinking

Need to start with some base functions that will get called a ton, these will include:

- Open, navigate, pull single year player game log (input player, year, output raw HTML table)
- Take HTML table, parse into rough df that needs formatting
- Take rough df and clean, output single year player stats

Once this is ready, create list of players that played since 2000, could be its own table? (player name, first year, last year)

For now lets use LBJ as our first player to put this in practice on

In [5]:
first = "Lebron"
last = "James"

In [7]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [28]:
# TODO extract into player specific helpers/util


def get_player_season_games_log(first, last, season):
    player_code = player_name_to_bbref_code(first, last)
    url = f"https://www.basketball-reference.com/players/{player_code}/gamelog/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_basic')
    df = pd.read_html(str(table))[0]
    return df

In [33]:
# TODO extract into selenium specific helper/util

# Options for selenium
options = Options()
options.page_load_strategy = 'eager'  # Faster load so it does not wait for video ads to render
options.add_argument("--headless")  # Run Chrome in headless mode

def get_webpage_html(url):
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    html = driver.page_source
    driver.close()
    return html

In [266]:
df = get_player_season_games_log(first, last, 2022)

In [267]:
df = clean_player_games_log(df)

KeyError: 1

In [40]:
df_original = df.copy()

In [268]:
# Specific for the player game log table

# TODO, investigate try/catch for this in case missing values?
def column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'FG'] = df['FG'].astype(int)
    df.loc[:, 'FGA'] = df['FGA'].astype(int)
    df.loc[:, 'FG%'] = df['FG%'].astype(float)
    df.loc[:, '3P'] = df['3P'].astype(int)
    df.loc[:, '3PA'] = df['3PA'].astype(int)
    df.loc[:, '3P%'] = df['3P%'].astype(float)
    df.loc[:, 'FT'] = df['FT'].astype(int)
    df.loc[:, 'FTA'] = df['FTA'].astype(int)
    df.loc[:, 'FT%'] = df['FT%'].astype(float)
    df.loc[:, 'ORB'] = df['ORB'].astype(int)
    df.loc[:, 'DRB'] = df['DRB'].astype(int)
    df.loc[:, 'TRB'] = df['TRB'].astype(int)
    df.loc[:, 'AST'] = df['AST'].astype(int)
    df.loc[:, 'STL'] = df['STL'].astype(int)
    df.loc[:, 'BLK'] = df['BLK'].astype(int)
    df.loc[:, 'TOV'] = df['TOV'].astype(int)
    df.loc[:, 'PF'] = df['PF'].astype(int)
    df.loc[:, 'PTS'] = df['PTS'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    # df.loc[:, 'PM'] = df['PM'].astype(int)
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)    
    # if type(df.date[1]) == str:
    df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [137]:
col_order = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 
             'Opp', 'home','result', 'score_diff', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', 
             '3PA', '3P%', 'FT','FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 
             'TOV', 'PF','PTS', 'GmSc', 'PM',] # Specific column order because cleaner

home_map = {np.nan: 1, "@": 0}

def extract_date(dt): # To create gameID column
    return dt.date().strftime("%Y%m%d")

def clean_player_games_log(df):
    
    df = df[df['G'] != 'G'].copy() # remove formatting rows
    column_rename = {'Rk':'tm_game_num', 'G':'p_game_num', 'Date': 'date', 'Age':'age','Unnamed: 5':'home', 'Unnamed: 7':'result','+/-':'PM'} 
    df.rename(columns=column_rename, inplace=True) # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy() # Remove rows where the player was inactive for the game
    df = df[df['GS'] != 'Did Not Dress'].copy() # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy() # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy() # Remove rows where player was not with team?


    # NOTE: Will be more of these to drop, e.g. did not dress
    
    df.loc[:, 'home'] = df['home'].replace(home_map) # Map home to a bool column, true = home

    # Split compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')

    df = df.drop(columns = ['age'])

    
    df = column_type_conversion(df)

    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]

    df = df[col_order]
    return df
    

In [124]:
df = df_original.copy()

In [125]:
df = clean_player_games_log(df)

In [269]:
# Test to import all lebron james seasons to single df

df = pd.DataFrame()
first = "Pascal"
last = "Siakam"

for year in range(2017, 2024):
    temp = get_player_season_games_log(first, last, year)
    temp = clean_player_games_log(temp)
    df = pd.concat([df, temp])  
    print('appended ', year)

df

appended  2017
appended  2018
appended  2019
appended  2020
appended  2021
appended  2022
appended  2023


,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,Opp,home,result,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,PM
0,20161026TORDET,1,1,2016-10-26 00:00:00,22,207,TOR,DET,True,W,...,7,9,0,1,0,0,2,4,7.1,+16
1,20161028TORCLE,2,2,2016-10-28 00:00:00,22,209,TOR,CLE,True,L,...,0,3,0,1,0,2,3,4,1.9,-7
2,20161031TORDEN,3,3,2016-10-31 00:00:00,22,212,TOR,DEN,True,W,...,3,4,0,0,0,0,3,2,0.0,-4
3,20161102TORWAS,4,4,2016-11-02 00:00:00,22,214,TOR,WAS,False,W,...,3,3,0,1,0,1,3,4,1.7,-6
4,20161104TORMIA,5,5,2016-11-04 00:00:00,22,216,TOR,MIA,True,W,...,2,3,0,0,0,1,0,8,5.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20230331TORPHI,77,67,2023-03-31 00:00:00,28,363,TOR,PHI,False,L,...,4,4,7,0,0,2,2,14,8.4,NaN
80,20230402TORCHO,78,68,2023-04-02 00:00:00,29,0,TOR,CHO,False,W,...,5,7,7,2,0,0,1,36,35.0,+16
81,20230404TORCHO,79,69,2023-04-04 00:00:00,29,2,TOR,CHO,False,W,...,11,14,4,2,2,2,4,22,21.6,+5
82,20230405TORBOS,80,70,2023-04-05 00:00:00,29,3,TOR,BOS,False,L,...,9,11,4,2,0,1,3,28,26.8,+1


In [146]:
df_lebron = df.copy()

In [147]:
# Test to import all Jokic seasons to single df

df = pd.DataFrame()
first = "Nikola"
last = "Jokic"

for year in range(2016, 2024):
    temp = get_player_season_games_log(first, last, year)
    temp = clean_player_games_log(temp)
    df = pd.concat([df, temp])  
    print('appended ', year)

df_jokic = df.copy()
df_jokic

appended  2016
appended  2017
appended  2018
appended  2019
appended  2020
appended  2021
appended  2022
appended  2023


,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,Opp,home,result,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,PM
0,20151028DENHOU,1,1,2015-10-28 00:00:00,20,251,DEN,HOU,False,W,...,0,0,0,0,0,1,0,2,0.7,+1
1,20151030DENMIN,2,2,2015-10-30 00:00:00,20,253,DEN,MIN,True,L,...,6,9,1,0,0,2,3,10,7.8,+10
2,20151101DENOKC,3,3,2015-11-01 00:00:00,20,255,DEN,OKC,False,L,...,1,4,0,0,0,1,2,8,3.9,-2
4,20151105DENUTA,5,4,2015-11-05 00:00:00,20,259,DEN,UTA,True,L,...,2,3,0,0,1,1,1,5,2.8,-6
5,20151106DENGSW,6,5,2015-11-06 00:00:00,20,260,DEN,GSW,False,L,...,2,4,1,1,1,1,1,4,4.2,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,20230322DENWAS,73,65,2023-03-22 00:00:00,28,31,DEN,WAS,False,W,...,7,12,7,3,0,2,1,31,35.7,+28
76,20230325DENMIL,74,66,2023-03-25 00:00:00,28,34,DEN,MIL,True,W,...,5,6,11,1,0,3,2,31,27.7,+19
77,20230327DENPHI,75,67,2023-03-27 00:00:00,28,36,DEN,PHI,True,W,...,14,17,12,0,2,3,3,25,31.6,+11
81,20230404DENHOU,79,68,2023-04-04 00:00:00,28,44,DEN,HOU,False,L,...,8,10,4,2,3,8,1,14,9.6,-21


In [270]:
df_pascal = df.copy()

In [284]:
pascal_recent = df_pascal[(df_pascal['date'] >= pd.to_datetime('2021-10-01')) & (df_pascal['home'])]

In [287]:
pascal_recent = df_pascal[(df_pascal['date'] >= pd.to_datetime('2022-10-01')) & (df_pascal['home'])]
pascal_bt = len(pascal_recent[pascal_recent['PTS'] >= 20])
pascal_total = pascal_recent.shape[0]
print(f"Pascal games over threshold {pascal_bt} out of {pascal_total}, above threshold {pascal_bt/pascal_total}% of the time")


Pascal games over threshold 27 out of 35, above threshold 0.7714285714285715% of the time


In [ ]:
jokic_bt = len(jokic_recent[jokic_recent['PTS'] > 20])
jokic_total = jokic_recent.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

In [154]:
jokic_recent = df_jokic[df_jokic['date'] >= pd.to_datetime('2021-01-01')]

In [155]:
jokic_bt = len(df_jokic[df_jokic['PTS'] < 15])
jokic_total = df_jokic.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

Jokic games under threshold 173 out of 596, below threshold 0.2902684563758389% of the time
Lebron games under threshold 59 out of 1421, below threshold 0.04152005629838142% of the time


In [156]:
jokic_bt = len(jokic_recent[jokic_recent['PTS'] < 15])
jokic_total = jokic_recent.shape[0]
lebron_bt = len(df_lebron[df_lebron['PTS'] < 15])
lebron_total = df_lebron.shape[0]
print(f"Jokic games under threshold {jokic_bt} out of {jokic_total}, below threshold {jokic_bt/jokic_total}% of the time")
print(f"Lebron games under threshold {lebron_bt} out of {lebron_total}, below threshold {lebron_bt/lebron_total}% of the time")

Jokic games under threshold 18 out of 211, below threshold 0.08530805687203792% of the time
Lebron games under threshold 59 out of 1421, below threshold 0.04152005629838142% of the time


## Scrape NBA Players list

Need to create a list of all NBA players that have played in the league since 2000, use the BBref player directory, pull each last name letter (A->Z), clean the table, will have list of all players in nba/aba history along with their first and last year in the league

In [179]:
def get_player_years_active(letter):
    url = f"https://www.basketball-reference.com/players/{letter}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'players')
    df = pd.read_html(str(table))[0]
    return df

In [224]:
def clean_players_table(df):
    df = df[df['From'] != 'From']
    cols_to_drop = ['Colleges', 'Birth Date', 'Wt', 'Ht', 'Pos']
    df = df.drop(columns = cols_to_drop)
    df[['first_name', 'last_name']] = df['Player'].str.split(' ', n=1, expand=True)
    df['last_name'] = df['last_name'].str.rstrip('*')
    df['From'] = df['From'].astype(int)
    df['To'] = df['To'].astype(int)
    df = df[['first_name', 'last_name','From','To']]
    return df

In [260]:
def clean_players_table_all(df):
    df = df[df['From'] != 'From'].copy()
    cols_to_rename = {'Birth Date': 'birth_date', 'Colleges': 'college'}
    df.rename(columns = cols_to_rename, inplace=True)
    df[['first_name', 'last_name']] = df['Player'].str.split(' ', n=1, expand=True)
    df['hall_of_fame'] = df['last_name'].str.endswith('*') # Keep track of hall of fame as a bool column
    df['last_name'] = df['last_name'].str.rstrip('*')
    df['From'] = df['From'].astype(int)
    df['To'] = df['To'].astype(int)
    df['Wt'] = df['Wt'].astype(int)
    if type(df.birth_date[1]) == str:
        df.loc[:, 'birth_date'] = pd.to_datetime(df['birth_date'], format='mixed')
    df = df[['first_name', 'last_name', 'From', 'To', 'Pos', 'Ht', 'Wt', 'birth_date', 'college', 'hall_of_fame']]
    return df


In [250]:
test = get_player_years_active('b')

In [251]:
test

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,Chris Babb,2014,2014,G,6-5,225,"February 14, 1990","Penn State, Iowa State"
1,Luke Babbitt,2011,2018,F,6-9,225,"June 20, 1989",Nevada
2,Miloš Babić,1991,1992,C-F,7-0,240,"November 23, 1968",Tennessee Technological University
3,Johnny Bach,1949,1949,F-G,6-2,180,"July 10, 1924",Fordham
4,Dwayne Bacon,2018,2021,G,6-6,221,"August 30, 1995",Florida State
...,...,...,...,...,...,...,...,...
509,Will Bynum,2006,2015,G,6-0,185,"January 4, 1983","Arizona, Georgia Tech"
510,Walt Byrd,1970,1970,F,6-7,205,NaN,Temple
511,Marty Byrnes,1979,1983,F,6-7,215,"April 30, 1956",Syracuse
512,Tommy Byrnes,1947,1951,F-G,6-3,175,"February 19, 1923",Seton Hall


In [261]:
df = test.copy()
df = clean_players_table_all(df)

In [262]:
df[df['hall_of_fame'] == True]

,first_name,last_name,From,To,Pos,Ht,Wt,birth_date,college,hall_of_fame
49,Charles,Barkley,1985,2000,F,6-6,252,1963-02-20 00:00:00,Auburn,True
51,Don,Barksdale,1952,1955,F-C,6-6,200,1923-03-31 00:00:00,UCLA,True
78,Rick,Barry,1966,1980,F,6-7,205,1944-03-28 00:00:00,Miami (FL),True
107,Elgin,Baylor,1959,1972,F,6-5,225,1934-09-16 00:00:00,"College of Idaho, Seattle University",True
123,Zelmo,Beaty,1963,1975,C,6-9,225,1939-10-25 00:00:00,Prairie View A&M University,True
144,Walt,Bellamy,1962,1975,C,6-11,225,1939-07-24 00:00:00,Indiana,True
191,Dave,Bing,1967,1978,G,6-3,180,1943-11-24 00:00:00,Syracuse,True
196,Larry,Bird,1980,1992,F,6-9,220,1956-12-07 00:00:00,Indiana State University,True
278,Chris,Bosh,2004,2016,F-C,6-11,235,1984-03-24 00:00:00,Georgia Tech,True
316,Bill,Bradley,1968,1977,F-G,6-5,205,1943-07-28 00:00:00,Princeton,True


In [185]:
players_df = pd.DataFrame()

for i in range(ord('a'), ord('z')+1):
    if(chr(i) == 'x'):
        continue
    temp = get_player_years_active(chr(i))
    players_df = pd.concat([players_df, temp])
    num = random.randint(1, 3) # Otherwise might have basketball ref find out you are a bot
    time.sleep(num)

players_df = clean_players_table(players_df)
players_df = players_df[(players_df['From'] > 2000) | (players_df['To'] > 2000)]
players_df

a
sleeping 4
b
sleeping 5
c
sleeping 1
d
sleeping 2
e
sleeping 1
f
sleeping 2
g
sleeping 4
h
sleeping 1
i
sleeping 3
j
sleeping 2
k
sleeping 2
l
sleeping 2
m
sleeping 5
n
sleeping 3
o
sleeping 5
p
sleeping 3
q
sleeping 5
r
sleeping 3
s
sleeping 5
t
sleeping 5
u
sleeping 2
v
sleeping 1
w
sleeping 4
x
y
sleeping 2
z
sleeping 3


## Start Scraping NBA Player Logs

#### Have completed so far
- Scrape game log for given player + season combo
- Script to pull all NBA players, create df -> csv of all players that played in 2000s

#### Need To Complete
- Investigate advanced stats appending
- For loop logic, open csv, for each player create list of years to iterate over, scrape each year for given player, append to DF

## Scrape Advanced Stats Games Log

- Pull for single player (done)
- Clean DF (done)
- Function to pull both
- Function to concat df together

In [288]:
# TODO extract into player specific helpers/util

def player_name_to_bbref_code(first, last):
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [293]:
# TODO extract into player specific helpers/util


def get_player_adv_season_games_log(first, last, season):
    player_code = player_name_to_bbref_code(first, last)
    url = f"https://www.basketball-reference.com/players/{player_code}/gamelog-advanced/{season}"
    html = get_webpage_html(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_advanced')
    df = pd.read_html(str(table))[0]
    return df

In [304]:
df2.columns

Index(['tm_game_num', 'p_game_num', 'date', 'Tm', 'home', 'Opp', 'result',
       'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM', 'age_yrs',
       'age_days', 'score_diff'],
      dtype='object')

In [305]:
# Specific for the player game log table

# TODO, investigate try/catch for this in case missing values?
def adv_column_type_conversion(df):
    df.loc[:, 'tm_game_num'] = df['tm_game_num'].astype(int)
    df.loc[:, 'p_game_num'] = df['p_game_num'].astype(int)
    df.loc[:, 'home'] = df['home'].astype(bool)
    df.loc[:, 'GS'] = df['GS'].astype(bool)
    df.loc[:, 'TS%'] = df['TS%'].astype(float)
    df.loc[:, 'eFG%'] = df['eFG%'].astype(float)
    df.loc[:, 'ORB%'] = df['ORB%'].astype(float)
    df.loc[:, 'DRB%'] = df['DRB%'].astype(float)
    df.loc[:, 'TRB%'] = df['TRB%'].astype(float)
    df.loc[:, 'AST%'] = df['AST%'].astype(float)
    df.loc[:, 'STL%'] = df['STL%'].astype(float)
    df.loc[:, 'BLK%'] = df['BLK%'].astype(float)
    df.loc[:, 'TOV%'] = df['TOV%'].astype(float)
    df.loc[:, 'USG%'] = df['USG%'].astype(float)
    df.loc[:, 'ORtg'] = df['ORtg'].astype(int)
    df.loc[:, 'DRtg'] = df['DRtg'].astype(int)
    df.loc[:, 'GmSc'] = df['GmSc'].astype(float)
    df.loc[:, 'BPM'] = df['BPM'].astype(float)
    df.loc[:, 'age_yrs'] = df['age_yrs'].astype(int)
    df.loc[:, 'age_days'] = df['age_days'].astype(int)
    df.loc[:, 'score_diff'] = df['score_diff'].astype(int)    
    if type(df.date[1]) == str:
        df.loc[:, 'date'] = pd.to_datetime(df['date'], format='mixed')
    return df

In [309]:
first = "Lebron"
last = "James"
season = 2005

df = get_player_adv_season_games_log(first, last, season)
df = clean_player_adv_games_log(df)
df

,gameID,tm_game_num,p_game_num,date,age_yrs,age_days,Tm,home,Opp,result,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
0,20041103CLEIND,1,1,2004-11-03 00:00:00,19,309,CLE,True,IND,L,...,6.3,33.5,1.1,4.6,19.8,30.3,103,105,20.9,7.4
1,20041104CLEMIA,2,2,2004-11-04 00:00:00,19,310,CLE,False,MIA,L,...,8.5,38.9,1.3,0.0,4.3,26.0,107,109,17.8,9.7
2,20041106CLEMIL,3,3,2004-11-06 00:00:00,19,312,CLE,False,MIL,L,...,10.4,32.9,3.6,0.0,11.9,34.6,104,103,23.8,9.2
3,20041109CLEATL,4,4,2004-11-09 00:00:00,19,315,CLE,False,ATL,W,...,13.9,17.4,4.7,3.2,11.9,25.8,111,83,22.5,8.7
4,20041110CLEPHO,5,5,2004-11-10 00:00:00,19,316,CLE,True,PHO,W,...,10.6,28.0,3.1,4.1,9.3,29.7,130,95,36.1,16.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,20050414CLENYK,78,76,2005-04-14 00:00:00,20,105,CLE,True,NYK,L,...,22.4,28.6,2.6,3.9,11.4,36.3,98,97,19.2,8.7
81,20050415CLEWAS,79,77,2005-04-15 00:00:00,20,106,CLE,False,WAS,L,...,7.5,23.1,1.1,0.0,2.7,32.1,121,131,25.7,5.5
82,20050417CLEDET,80,78,2005-04-17 00:00:00,20,108,CLE,False,DET,L,...,12.5,31.7,0.0,1.5,2.4,42.6,110,111,24.5,14.3
84,20050419CLEBOS,81,79,2005-04-19 00:00:00,20,110,CLE,True,BOS,W,...,10.5,22.9,5.8,0.0,9.3,31.0,121,92,25.7,8.4


In [297]:
df.columns

Index(['Rk', 'G', 'Date', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7', 'GS',
       'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'],
      dtype='object')

In [ ]:
['Rk', 'G', 'Date', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7', 'GS',
       'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'

In [308]:
col_order = ['gameID', 'tm_game_num', 'p_game_num', 'date', 'age_yrs', 'age_days', 'Tm', 'home', 'Opp', 
             'result', 'score_diff', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 
             'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'] # Specific column order because cleaner

home_map = {np.nan: 1, "@": 0}

def extract_date(dt): # To create gameID column
    return dt.date().strftime("%Y%m%d")

def clean_player_adv_games_log(df):
    
    df = df[df['G'] != 'G'].copy() # remove formatting rows
    column_rename = {'Rk':'tm_game_num', 'G':'p_game_num', 'Date': 'date', 'Age':'age','Unnamed: 5':'home', 'Unnamed: 7':'result'} 
    df.rename(columns=column_rename, inplace=True) # Rename necessary columns

    df = df[df['GS'] != 'Inactive'].copy() # Remove rows where the player was inactive for the game
    df = df[df['GS'] != 'Did Not Dress'].copy() # Remove rows where player was not active (for player stats research would want to keep, but for ML not needed
    df = df[df['GS'] != 'Did Not Play'].copy() # Remove rows where player was not active, mutual decision? lol
    df = df[df['GS'] != 'Not With Team'].copy() # Remove rows where player was not with team?


    # NOTE: Will be more of these to drop, e.g. did not dress
    
    df.loc[:, 'home'] = df['home'].replace(home_map) # Map home to a bool column, true = home

    # Split compound columns
    df[['age_yrs', 'age_days']] = df['age'].str.split('-', expand=True)
    df[['result', 'score_diff']] = df['result'].str.split('(', expand=True)
    df['score_diff'] = df['score_diff'].str.rstrip(')')

    df = df.drop(columns = ['age'])

    
    df = adv_column_type_conversion(df)

    df['gameID'] = df['date'].apply(extract_date) + df["Tm"] + df["Opp"]

    df = df[col_order]
    return df
    

In [301]:
df2 = clean_player_adv_games_log(df)